#### Part 1 - Defining class for BPNN

In [1]:
import numpy as np
import pandas as pd

In [2]:
class BPNN:
    def __init__(
        self,hidden_layer_sizes,
        l_rate=.001,
        n_epoch=20,
        batch_size=20,
        random_state=0
    ):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.l_rate= float(l_rate)
        self.n_epoch = int(n_epoch)
        self.batch_size = int(batch_size)
        self.random_state = random_state
        
        self.n = len(hidden_layer_sizes)+1
        self.outputs = [None]*(self.n+1)
        self.delta = [None]*self.n
        
        
    def activation(self,x):
        return 1/(1+np.exp(-x))
    
    def d_activation(self,x):
         return x * (1 - x)
    
    def pad_ones(X):
        pad_width = [(1,0),(0,0)]
        return np.pad(X,pad_width=pad_width,constant_values=1)
    
    def inputs(self,i):
        return BPNN.pad_ones(self.outputs[i-1])
    
    def initialize_random_weights(self):
        np.random.seed(self.random_state)
        self.weights = [
            np.random.standard_normal((o,i+1))*.001
            for i,o in zip(
                [self.n_inputs]+self.hidden_layer_sizes,
                self.hidden_layer_sizes+[self.n_outputs]
            )
        ]

    def forward_propagate(self, inputs):
        assert (self.weights is not None),"weights not given"
        self.outputs[-1] = inputs
        for i in range(self.n):
            self.outputs[i] = self.activation(
                self.weights[i] @ self.inputs(i)
            )
        return self.outputs[self.n-1]
    
    def backward_propagate_error(self, target):
        for i in range(self.n-1, -1, -1):
            if i == self.n-1:
                errors = self.outputs[i] - target
            else:
                errors = self.weights[i+1][:,1:].T @ self.delta[i+1] 
            self.delta[i] = errors * self.d_activation(self.outputs[i])

    def update_weights(self):
        for i in range(self.n):
            self.weights[i] -= self.l_rate * self.delta[i] @ self.inputs(i).T/self.batch_size
        
    def error(self,actual,predicted):
        error = actual-predicted
        return np.sum(error * error)
    
    def fit(self, X_train,y_train,verbose=False):
        m = X_train.shape[0]
        input_matrix = X_train.T
        classes = np.unique(y_train)
        target_matrix = (y_train.reshape(-1,1) == classes).T
        self.n_inputs = input_matrix.shape[0]
        self.n_outputs = target_matrix.shape[0]
        self.initialize_random_weights()
        if verbose:
            print("Initial Weights:")
            print(*self.weights,sep="\n")
            print("Training:")
            
        for epoch in range(self.n_epoch):
            sum_error = 0
            fs = range(m+self.batch_size)
            for f,t in zip(fs,fs[1:]):
                outputs = self.forward_propagate(input_matrix[:,f:t])
                sum_error += self.error(target_matrix[:,f:t],outputs)
                self.backward_propagate_error(target_matrix[:,f:t])
                self.update_weights()
            if verbose:
                print(f'> epoch={epoch+1}, lrate={self.l_rate:.3}, error={sum_error/m:.5f}')
        if verbose:
            print("Trained Weights:")
            print(*self.weights,sep="\n")
            
    def predict(self, inputs):
        outputs = self.forward_propagate(inputs.T).T.argmax(axis=-1)
        return outputs

    def accuracy(self,X_test,y_test):
        return (self.predict(X_test)==y_test).mean()

#### Part 2 - Loading and processing dataset

In [3]:
titanic_df = pd.read_csv("datasets/titanic_processed.csv")
X = titanic_df.drop('Survived',axis = 1).values
y = titanic_df['Survived'].values
split_ratio = 0.8
s = int(split_ratio*X.shape[0])
X_train, X_test = X[:s],X[s:]
y_train, y_test = y[:s],y[s:]

#### Part 3 - Implementing BPNN

In [4]:
b = BPNN(
    hidden_layer_sizes = [5,3],
    l_rate=30, n_epoch=20, batch_size=25,random_state=10
)
b.fit(X_train,y_train,verbose=True)
print("Evaluation:")
print(f"Acuracy of the classifier: {b.accuracy(X_test,y_test)*100:.2f}%")

Initial Weights:
[[ 1.33158650e-03  7.15278974e-04 -1.54540029e-03 -8.38384993e-06
   6.21335974e-04 -7.20085561e-04  2.65511586e-04  1.08548526e-04
   4.29143093e-06 -1.74600211e-04]
 [ 4.33026190e-04  1.20303737e-03 -9.65065671e-04  1.02827408e-03
   2.28630130e-04  4.45137613e-04 -1.13660221e-03  1.35136878e-04
   1.48453700e-03 -1.07980489e-03]
 [-1.97772828e-03 -1.74337230e-03  2.66070164e-04  2.38496733e-03
   1.12369125e-03  1.67262221e-03  9.91492158e-05  1.39799638e-03
  -2.71247988e-04  6.13204185e-04]
 [-2.67317189e-04 -5.49309014e-04  1.32708296e-04 -4.76142015e-04
   1.30847308e-03  1.95013279e-04  4.00209988e-04 -3.37632337e-04
   1.25647226e-03 -7.31969502e-04]
 [ 6.60231551e-04 -3.50871891e-04 -9.39433360e-04 -4.89337217e-04
  -8.04591142e-04 -2.12697639e-04 -3.39140246e-04  3.12169936e-04
   5.65152670e-04 -1.47420258e-04]]
[[-2.59053368e-05  2.89094204e-04 -5.39879071e-04  7.08160020e-04
   8.42224738e-04  2.03580797e-04]
 [ 2.39470366e-03  9.17458938e-04 -1.12272471e